In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import requests
import folium
# ! pip install pandas => 파이썬 데이터 처리 라이브러리
import pandas as pd
from pandas.io.json import json_normalize # 파이썬 딕셔너리 타입의 데이터를 pandas 데이터 프레임으로 변환한다.

# 전국 또는 특정 지역의 스타벅스 매장 위치 정보를 가져와 지도 위에 표시한다.
requests로 스타벅스 매장의 위치 정보를 가져와서 딕셔너리 타입으로 변환한다.
json_normalize() 함수로 json 타입의 데이터가 변환된 파이썬 딕셔너리를 판다스 데이터프레임으로 변환한다.
folium 모듈을 사용해서 지도를 표시하고, 지도 위에 스타벅스 매장 위치를 마커로 표시한다.

In [11]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=P2QVWUB9H1'
req = requests.post(targetSite, data={
    'ins_lat': 37.5717888, 
    'ins_lng': 126.9661696, 
    'p_sido_cd' : '01', 
    'p_gugun_cd' : '', 
    'in_biz_cd' : '',
    'iend': 2000,
    'set_date' : ''
})
starbucks = req.json()

In [13]:
# pandas.io.json 모듈의 json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_normalize(딕셔너리, 데이터프레임으로 변환할 데이터가 할당된 딕셔너리의 key)
starbucks_df = json_normalize(starbucks, 'list')
print(type(starbucks_df)) # <class 'pandas.core.frame.DataFrame'>
print(len(starbucks_df))

<class 'pandas.core.frame.DataFrame'>
599


In [14]:
starbucks_df

seq sido_cd sido_nm gugun_cd gugun_nm code_order view_yn store_num  sido  \
0      0    None    None     None     None       None    None      None  None   
1      0    None    None     None     None       None    None      None  None   
2      0    None    None     None     None       None    None      None  None   
3      0    None    None     None     None       None    None      None  None   
4      0    None    None     None     None       None    None      None  None   
..   ...     ...     ...      ...      ...        ...     ...       ...   ...   
594    0    None    None     None     None       None    None      None  None   
595    0    None    None     None     None       None    None      None  None   
596    0    None    None     None     None       None    None      None  None   
597    0    None    None     None     None       None    None      None  None   
598    0    None    None     None     None       None    None      None  None   

    gugun  ... t22 t21  p90 t05 t30 t36 t27 t29 t43 t48  
0    None  ...   0   0    0   0   0   0   0   0   0   0  
1    None  ...   0   0    0   0   0   0   0   0   0   0  
2    None  ...   0   0    0   0   0   0   0   0   0   0  
3    None  ...   0   0    0   0   0   0   0   0   0   0  
4    None  ...   0   0    0   0   0   0   0   0   0   0  
..    ...  ...  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..  
594  None  ...   0   0    0   0   0   0   0   0   0   0  
595  None  ...   0   0    0   0   0   0   0   0   0   0  
596  None  ...   0   0    0   0   0   0   0   0   0   0  
597  None  ...   0   0    0   0   0   0   0   0   0   0  
598  None  ...   0   0    0   0   0   0   0   0   0   0  

[599 rows x 129 columns]

In [ ]:
# 변환 결과 서울의 스타벅스 지점 599개 * 지점당 129개의 컬럼으로 정보가 표시된다.
# 작업에 필요한 일부 컬럼을 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name        => 지점 이름
# sido_code     => 시도 코드
# sido_name     => 시도 이름
# gugun_code    => 구군 코드
# gugun_name    => 구군 이름
# doro_address  => 도로명주소
# lat           => 위도
# lot           => 경도

In [55]:
# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 얻어올 컬럼이 1개면 컬럼 이름만 적고, 얻어올 컬럼이 2개 이상이면 얻어올 컬럼들을 리스트로 만들어서 얻어온다.
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'lat', 'lot', 'doro_address']]
starbucks_df_map.head()

s_name sido_code sido_name gugun_code gugun_name        lat         lot  \
0      서대문역        01        서울       0118        종로구   37.56670   126.96629   
1   서울대치과병원        01        서울       0118        종로구   37.57790   126.99965   
2     동숭길입구        01        서울       0118        종로구  37.584236  127.002014   
3  강북삼성병원1층        01        서울       0118        종로구  37.568546   126.96756   
4    종로플레이스        01        서울       0118        종로구   37.57319  126.998309   

                        doro_address  
0             서울특별시 종로구 통일로 134 (평동)  
1  서울특별시 종로구 대학로 101 서울대학교 치과병원 신관1층  
2            서울특별시 종로구 대학로 146 (혜화동)  
3             서울특별시 종로구 새문안로 29 (평동)  
4           서울특별시 종로구 창경궁로 120 (인의동)

In [54]:
# dtypes 속성으로 판다스 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
# 판다스는 문자열 데이터 타입을 object라 부른다.
starbucks_df_map.dtypes

s_name           object
sido_code        object
sido_name        object
gugun_code       object
gugun_name       object
lat             float64
lot             float64
doro_address     object
dtype: object

In [53]:
# info() 메소드를 사용해도 데이터 프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   s_name        40 non-null     object 
 1   sido_code     40 non-null     object 
 2   sido_name     40 non-null     object 
 3   gugun_code    40 non-null     object 
 4   gugun_name    40 non-null     object 
 5   lat           40 non-null     float64
 6   lot           40 non-null     float64
 7   doro_address  40 non-null     object 
dtypes: float64(2), object(6)
memory usage: 2.6+ KB


In [52]:
# astype() 메소드로 위도, 경도의 데이터 타입을 object에서 float로 변환한다.
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   s_name        40 non-null     object 
 1   sido_code     40 non-null     object 
 2   sido_name     40 non-null     object 
 3   gugun_code    40 non-null     object 
 4   gugun_name    40 non-null     object 
 5   lat           40 non-null     float64
 6   lot           40 non-null     float64
 7   doro_address  40 non-null     object 
dtypes: float64(2), object(6)
memory usage: 2.6+ KB


In [29]:
# 종각점
starbucks_df_map[starbucks_df_map['s_name'] == '종각']

s_name sido_code sido_name gugun_code gugun_name        lat         lot
439     종각        01        서울       0118        종로구  37.569918  126.984528

In [30]:
# 종로관철점
starbucks_df_map[starbucks_df_map['s_name'] == '종로관철']

s_name sido_code sido_name gugun_code gugun_name        lat         lot
441   종로관철        01        서울       0118        종로구  37.569058  126.986013

In [33]:
# 종로구청점
starbucks_df_map[starbucks_df_map['s_name'] == '종로구청']

s_name sido_code sido_name gugun_code gugun_name        lat         lot
426   종로구청        01        서울       0118        종로구  37.572662  126.981138

In [ ]:
starbucksMap = folium.Map(location=[37.569918, 126.984528], zoom_start=18)
popup = folium.Popup('종각점', max_width=300)
folium.Marker(location=[37.569918, 126.984528], popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로관철점', max_width=300)
folium.Marker(location=[37.569058, 126.986013], popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로구청점', max_width=300)
folium.Marker(location=[37.572662, 126.981138], popup=popup).add_to(starbucksMap)
starbucksMap

In [56]:
# 서울특별시에 위치한 스타벅스 전 지점의 위치를 지도 위에 표시한다.
# 지도를 작성할 때 지도의 중심으로 설정할 위치가 애매할 경우 지도 위에 표시할 모든 스타벅스 지점의 위도와 경도의 평균을 중심으로 하는 지도를 만들어 사용하면 된다.
# 판다스 데이터프레임에 저장된 데이터의 평균은 mean() 메소드를 사용해서 계산한다.
starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=17)
# iterrows() 메소드는 판다스 데이터 프레임에 저장된 index와 데이터를 리턴한다.
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {}: {}'.format(index + 1 , data['s_name'], data['doro_address']), max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMaps2.html')
starbucksMap

TypeError: Could not convert 37.5667037.5779037.58423637.56854637.5731937.56857837.57101737.56938537.57557837.57650937.570552737.5815429637.573863737.577150837.56905837.58296937.569918337.5715491437.5753221237.58289237.571229337.572129337.5752043137.571437.58538137.57733837.569659737.571493137.57946237.57266237.57270037.57441137.57143237.57080437.56985437.60614937.56854637.57175937.582477937.5743289 to numeric

In [58]:
# 서울특별시 종로구 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=P2QVWUB9H1'
req = requests.post(targetSite, data={
    'ins_lat': 37.5717888, 
    'ins_lng': 126.9661696, 
    'p_sido_cd' : '', 
    'p_gugun_cd' : '0118', 
    'in_biz_cd' : '',
    'iend': 100,
    'set_date' : ''
})
starbucks = req.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'lat', 'lot', 'doro_address']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=16)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {}: {}'.format(index + 1 , data['s_name'], data['doro_address']), max_width=300)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMaps3.html')
starbucksMap